In [2]:
from urllib2 import urlopen
from json import load
import pandas as pd
from pandas.io.json import json_normalize
import itertools

In [3]:
##api_key = 'MDIzNDI5NDMzMDE0NTkwMTI5ODY1NzQ4Ng000'

In [4]:
url = 'http://api.npr.org/query?apiKey=MDIzNDI5NDMzMDE0NTkwMTI5ODY1NzQ4Ng000&numResults=3&format=json&id=1002'

In [5]:
response = urlopen(url)
json_obj = load(response)

In [6]:
##the next several cells are test code to practice mapping to the information we want in the df

In [7]:
'''
for story in json_obj['list']['story']:

    print 'ID: ' + story['id']
    
    print 'TITLE: ' + story['title']['$text']

    print 'DATE: ' + story['storyDate']['$text'] + '\n'
'''

"\nfor story in json_obj['list']['story']:\n\n    print 'ID: ' + story['id']\n    \n    print 'TITLE: ' + story['title']['$text']\n\n    print 'DATE: ' + story['storyDate']['$text'] + '\n'\n"

In [8]:
'''for story in json_obj['list']['story']:
    for parent in story['parent']:
        print story['id'], parent['title']['$text']
'''

"for story in json_obj['list']['story']:\n    for parent in story['parent']:\n        print story['id'], parent['title']['$text']\n"

In [9]:
'''for story in json_obj['list']['story']:
    print story['id'], story['text']['paragraph']
'''
'''      
for story in json_obj['list']['story']:
    for text in story['text']['paragraph']:
        print story['id'], text['$text']
'''

"      \nfor story in json_obj['list']['story']:\n    for text in story['text']['paragraph']:\n        print story['id'], text['$text']\n"

In [10]:
## Build the data frame by creating a list of dictionaries, then converting the list of dictionaries into a data frame

##First data frame: one row per distinct story category

dicts_list = []

In [11]:
for story in json_obj['list']['story']:
    for parent in story['parent']:
        d = {
            'id': story['id'],
            'title': story['title']['$text'],
            'category': parent['title']['$text']    
        }
        dicts_list.append(d)

In [12]:
df = pd.DataFrame(dicts_list, columns=('id', 'title', 'category'))

In [13]:
df.head(10)

,id,title,category
0,477094896,Revisiting A Mother's Day Story Of Separation,Around the Nation
1,477094896,Revisiting A Mother's Day Story Of Separation,"All Things Considered for May 8, 2016"
2,477094896,Revisiting A Mother's Day Story Of Separation,Mother's Day
3,477094896,Revisiting A Mother's Day Story Of Separation,rehabilitation
4,477094896,Revisiting A Mother's Day Story Of Separation,District of Columbia
5,477094896,Revisiting A Mother's Day Story Of Separation,Divorce
6,477094896,Revisiting A Mother's Day Story Of Separation,Around the Nation
7,477094896,Revisiting A Mother's Day Story Of Separation,U.S.
8,477094896,Revisiting A Mother's Day Story Of Separation,Home Page Top Stories
9,477094896,Revisiting A Mother's Day Story Of Separation,News


In [14]:
##Second data frame: one row per story, with one column containing a list of all the story's categories

dicts_reshape = []

In [15]:
for story in json_obj['list']['story']:
    categories_list = []
    d = {
        'id': story['id'],
        'title': story['title']['$text']
        }
    for parent in story['parent']:
        category = parent['title']['$text'].encode('ascii')
        categories_list.append(category)   
        d['category'] = categories_list
    dicts_reshape.append(d)

In [16]:
df_reshape = pd.DataFrame(dicts_reshape, columns=('id', 'title', 'category'))

In [17]:
df_reshape

,id,title,category
0,477094896,Revisiting A Mother's Day Story Of Separation,"[Around the Nation, All Things Considered for ..."
1,473397042,"Radio Drama's Leading Man, Still Adventuring","[Digital Life, All Things Considered for May 8..."
2,477046275,Is American Oil 'Dead'? T. Boone Pickens Says ...,"[Business, All Things Considered for May 8, 20..."


In [18]:
##df_reshape['category'][0]

In [19]:
##d3 export:  create a list of pairs of relationships to convert to json

pairs_d3 = []

In [20]:
for category_list in df_reshape['category']:
    for pair in itertools.combinations(category_list, 2):
        pairs_d3.append(pair)

In [21]:
df_d3 = pd.DataFrame(pairs_d3, columns=('source', 'target'))

In [22]:
df_d3

,source,target
0,Around the Nation,"All Things Considered for May 8, 2016"
1,Around the Nation,Mother's Day
2,Around the Nation,rehabilitation
3,Around the Nation,District of Columbia
4,Around the Nation,Divorce
5,Around the Nation,Around the Nation
6,Around the Nation,U.S.
7,Around the Nation,Home Page Top Stories
8,Around the Nation,News
9,Around the Nation,All Things Considered


In [23]:
df_d3.to_csv('d3_test.csv')